## Import Libraries

In [23]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split 
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.cross_validation import cross_val_predict
import operator
import csv

## Load general data

In [24]:
path  = '/Users/andremendes/OneDrive/Estudar/Estudar Projects/JFS/Data-Analysis-JFS-Matlab-Python/Data/'
scores = pd.read_csv(path+"text_scores.csv")
scores = scores.iloc[:,1:]
general = pd.read_csv(path+'numerical_data_csv.csv')
data = general.iloc[:,2:14]
categories = data.columns.values

## Normalize

In [25]:
minmax_scale = preprocessing.MinMaxScaler().fit(data)
df_minmax = minmax_scale.transform(data)
data_norm = pd.DataFrame(df_minmax, columns=categories)

## Split data

In [26]:
np.random.seed(5)
data_norm  = data_norm.reindex(np.random.permutation(data_norm.index))
data_norm, unknown = train_test_split(data_norm, test_size = 0.33)

## Prepare features, labels, train classifier and predict

In [27]:
y = np.array(data_norm.iloc[:,11:12])
y = y.ravel()
X = np.array(data_norm.iloc[:,:11])

saspe = LogisticRegression()
predicted = cross_val_predict(saspe, X, y, cv=10)
saspe.fit(X,y)
precision_recall_fscore_support(y, predicted, average='binary')

(0.61682242990654201, 0.81481481481481477, 0.70212765957446799, None)

## Classify using unknown data

In [28]:
y = np.array(unknown.iloc[:,11:12])
y = y.ravel()
X = unknown.iloc[:,:11]
features = list(X.columns.values)
X = np.array(X)

predicted = saspe.predict(X)
precision_recall_fscore_support(y, predicted, average='binary')

(0.51111111111111107, 0.74193548387096775, 0.60526315789473684, None)

In [29]:
coefficients = {}
weights = list(list(saspe.coef_[0]))

connection = zip(features,weights)
for feature, weight in connection:
    coefficients[feature] = weight

coefficients = sorted(coefficients.items(), key=operator.itemgetter(1))

coefficients

[('Age', -0.6713541293193187),
 ('School Year', -0.64914161961484718),
 ('Support for extra activities', -0.56802680036257391),
 ('GPA', -0.46947782007156469),
 ('Gender', -0.38646864699832029),
 ('Type School', -0.079765642582630608),
 ('English', 0.069319518381808401),
 ('Upload Transcript', 0.2558208187426616),
 ('know about Estudar', 0.28884109466362484),
 ('english percentil', 1.5476106936379548),
 ('english test', 3.4442442211754929)]

In [33]:
with open(path+'index_unknown.csv', 'wb') as f:
    writer = csv.writer(f)
    for val in unknown.index.tolist():
        writer.writerow([val])
        
with open(path+'index_known.csv', 'wb') as f:
    writer = csv.writer(f)
    for val in unknown.index.tolist():
        writer.writerow([val])